#### Building an Action Server

Once you have defined the action ([here](103e-ROS_action_definition.ipynb)) the next step is to write the python code that will define the server.  We first define the server, importing the message type we defined above and defining a callback that returns the result.  

In [ ]:
os.chdir(workspace)

In [ ]:
%%writefile timer_action_server.py

import time
import rclpy
from rclpy.action import ActionServer
from rclpy.node import Node

from timer.action import Timer

class TimerActionServer(Node):

    def __init__(self):
        super().__init__('timer_action_server')
        self._action_server = ActionServer(
            self,Timer,'timer',self.execute_callback)
        print('set up and waiting')

    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')
        feedback_msg = Timer.Feedback()
        duration = goal_handle.request.duration

        start = time.time()
        while (time.time()-start) < goal_handle.request.duration:
            feedback_msg.sec_left = duration - int(time.time()-start)
            self.get_logger().info('Feedback: {0}'.format(feedback_msg.sec_left))
            goal_handle.publish_feedback(feedback_msg)
            time.sleep(1)

        goal_handle.succeed()

        result = Timer.Result()
        result.done = 0
        return result

def main(args=None):
    rclpy.init(args=args)

    timer_action_server = TimerActionServer()

    rclpy.spin(timer_action_server)


if __name__ == '__main__':
    main()

Now you can run it in the terminal by typing:
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
python3 timer_action_server.py
```
Then open up a second terminal, and type:
```bash
cd ~/ros2_ws
export ROS_DOMAIN_ID=0
. install/setup.bash
ros2 action send_goal timer timer/action/Timer "{duration: 10}"
```

For some reason I cannot run this in the notebook

In [ ]:
os.chdir(workspace)
!. install/setup.bash
!export ROS_DOMAIN_ID=0
!pwd
!ros2 action send_goal timer timer/action/Timer "{duration: 10}"